In [ ]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
import csv
import dotenv
import os
from dotenv import load_dotenv, find_dotenv
from flipside import Flipside
import logging
# from degate.spot import Spot as Client
# from degate.lib.utils import config_logging
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
load_dotenv()
dotenv.load_dotenv('../../.env')
api_key = os.environ["DUNE_API_KEY"]
headers = {"X-Dune-API-Key": api_key}
flipside_key = os.environ["FLIPSIDE_API_KEY"]

In [ ]:
# fetch list of tokens registered on the platform from the flipside api
# https://docs.flipsidecrypto.xyz/data/data-products/api-sdk-developers/get-started-1/run-your-first-query
flipside = Flipside(flipside_key, "https://api-v2.flipsidecrypto.xyz")
sql = """with tokens as (
SELECT 
  DECODED_LOG: token as token
  , DECODED_LOG: tokenId as tokenId
  , *
FROM ethereum.core.ez_decoded_event_logs

where CONTRACT_ADDRESS = '0x9c07a72177c5a05410ca338823e790876e79d73b'
AND EVENT_NAME = 'TokenRegistered'
)

SELECT 
  DISTINCT token
  , tokenId
  , case 
      when token = '0x0000000000000000000000000000000000000000' then 'ETH'
      when token = '0xf2d4900994b24adac8b396500f24e0557e2bf84d' then 'RATS'
      when token = '0x32e4a492068bee178a42382699dbbe8eef078800' then 'BIRDPAW'
      when token = '0x4877d83faac234dbd38d0f17bbc27a80604d3aed' then 'Illiterate' --DeflationaryToken
      when token = '0x3f1ee2f15da3eaf3539006b651144ec87755876d' then 'BRETT' --deGate -- Brdg Token
      when token = '0x4b63ce7e179d1db5ddac5d9d54e48356cf3e8b7d' then 'AIB' -- on ERC20 V1 (AIB)
      when token = '0xa1cfa45c8313c2da73b93454adcb1dab24f0993a' then 'Elon' --Eloncoin 
      when token = '0xe97fb0268474ef444d732f4ce5cfa6d8772b97c6' then 'W' --Wormhole token 
      when token = '0x9aea32b459e96c8ef5010f69130bf95fd129ac05' then 'WKLAY' --Wrapped Klay  Wormhole -BT
      when token = '0x754e822b92ba9be681c461347e0d2d38abc0cf16' then 'LBS' --Lbscoin
      when token = '0xe31cfcd36fed044ae4cf9405b577fe875762194f' then 'ALGO'
      when token = '0xbf8c53c59fad2aff7ffd925db435ea10c5ea4b6c' then 'SIC' --Sicuro 
      when token = '0x061f60d153beeb3f78ed5f38bb326c5b20a65503' then 'ZPG' --zk pet dog 
      when token = '0x8cdf7af57e4c8b930e1b23c477c22f076530585e' then 'APT' --Aptos Coin (
      when token = '0x1c88d38d04acd3edd9051ec587c67abff04bf30d' then 'NEAR' --Bridget Token contract (BT)
      when token = '0x215cff9fa9f3466b07bc6b5a5f30b925fb71163b' then 'MR' -- Martin 
      when token = '0xccf27d3fff920d999cc7e8a3fc847a96bca44ccd' then 'SVR' -- SVR Token 
      when token = '0x366863c4d67f87cc1238d1008c7053f96e53e559' then 'Sailor'
      when token = '0x99613a517d20944246ab6ef124a735227a8c1af3' then 'Olda' --Oldacoin
      when token = '0xce0cd513a069e8ec9cb625fcdf6d5f29aa912dbc' then 'MMS'
      when token = '0x3635ffc4f860055a7f64365d39a27de3d84eb78b' then 'MAGA' --Magacoin (
      when token = '0x06b089cbf0403ac2c5f452584f8a18978019b858' then 'TBC' --Trade Bot Coin (
      when token = '0xb821b75b42da3c9f38383e457fa33c4e4b85a314' then 'GREN' -- Grencoin (
      when token = '0x6215a0fc6ba68cbb0f99a9e1d3a5adf1321b6eb7' then '$MUSK' --MUSK COIN 
      when token = '0xefc0ced4b3d536103e76a1c4c74f0385c8f4bdd3' then 'PYTH'--Pyth Network 
      when token = '0x84074ea631dec7a4edcd5303d164d5dea4c653d6' then 'SUI' --Bridget Token contract
      when token = '0xf91e605af079384cc7077b3914a4a36019a89ee8' then 'SEI' --Bridget Token contract
      when token = '0xf4feec8cf825cd5b23f8abb3075c01c22abd4352' then 'DEGEN' -- BT
      when token = '0x8a00bf67a9bb032204da83408c4d1cd5421b40b8' then 'DEGEN' --DEGEN Donkey 
      when token = '0x57fbf85655c3a08bffe37a4f32f8adbd369508df' then 'OLD' --Oldtown Coin (
      when token = '0x8f5affe2443ea12c575ad5b13bf8fd235ed184c9' then 'TEST' 
      when token = '0x4b63ce7e179d1db5ddac5d9d54e48356cf3e8b7d' then 'AIB' 
      when token = '0x256a63a4900bddcf7703b601ac0b70aa2d7f9318' then 'SORA'
      when token = '0xf852ffa34a20113cd741f3cd9406a1a86b70c8ab' then 'UNI-V2' -- SAGE/ETH pair
      when token = '0xf02123509a08632339102ee5fdd41b638592e495' then 'VEN' --DUCATO 
      when token = '0xf2fdd72bd1581b9bca7b4391975dbacca7ec37e8' then 'SHIB' ---SHIBcoin 
      when token = '0x2ed58b1fa208e9a08fdaac2a839b8539abe558e8' then 'WIF' --BT - dogwifhat 
      when token = '0x5e9b0c790707b95457d56fc8c6411662f61d4d98' then 'LYV' -- LYV Finance (
      when token = '0xe32e3851e0a4216581342defac353d1efdfb36d9' then 'SMFC' --Social Media Finance Coin (
      when token = '0x285308b5fc68cc0f737c77cb60042b1fb5633e81' then 'ZEUS' --Zeus Vip Coin 
      when token = '0x8ce949b02edc782c04f9c618396a9f8c0e2b9274' then 'ZBC' -- Zeus Basis Coin 
      when token = '0x1df721d242e0783f8fcab4a9ffe4f35bdf329909' then 'OP' -- BT Optimism 
      when token = '0x8687a10bca6f139b25eb31020fcabb5782214764' then 'JUP' --Jupiter 
      when token = '0x0a866a8256832aaf048f274b9b538e795b64137f' then 'NAP' --Napcoin 
      else symbol end as symbol
FROM tokens t
left join ethereum.price.ez_asset_metadata p ON t.token = p.token_address
"""

query_result_set = flipside.query(sql)
print(query_result_set)

In [ ]:
def auto_paginate_result(query_result_set, page_size=100):
    """
    Automatically paginates through the API results and returns all records.
    
    :param flipside: The API client instance
    :param query_id: The query ID for fetching results
    :param page_size: Number of records per page (default: 100)
    :return: A list of all records from paginated results
    """
    current_page_number = 1
    total_pages = 2  # Initial assumption until we get actual total pages
    all_records = []
    
    while current_page_number <= total_pages:
        try:
            results = flipside.get_query_results(
                query_result_set.query_id,
                page_number=current_page_number,
                page_size=page_size
            )

            # if current_page_number == 1 and results.records:
            #     headers = results.column_names  # Extract headers from first page
            
            # Update total pages from the response
            total_pages = results.page.totalPages
            
            # Append records if available
            if results.records:
                all_records.extend(results.records)
            
            # Move to the next page
            logging.info(f"Fetched page {current_page_number}/{total_pages}")
            current_page_number += 1
        
        except Exception as e:
            logging.error(f"Error fetching page {current_page_number}: {e}")
            break  # Exit loop in case of an error
    
    if all_records:
    # Save the results to a CSV file
        with open("token_metadata.csv", "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=all_records[0].keys())
            writer.writeheader()
            writer.writerows(all_records)
            print(f"Results saved to {f.name}")
        logging.info(f"Results saved to {f.name}")
    else:
        logging.warning("No records found in the results")
    return all_records

# print(auto_paginate_result(query_result_set))

THOUGHTS:
1. The part above is used one to create the token metadata file.
2. Now I need to write a function that will search for a new tokens and then add them to the metadata table and pass the new tokens to check which new pairs was created. 


In [ ]:
# 01/03 test, declare the function to fetch the registered tokens to autmate the token checking
from flipside import Flipside

def fetch_registered_tokens(flipside_key, contract_address):
    """
    Fetches the list of tokens registered on the platform from Flipside API.
    
    :param flipside_key: API key for FlipsideCrypto
    :param contract_address: The contract address to filter registered tokens
    :return: List of registered tokens
    """
    flipside = Flipside(flipside_key, "https://api-v2.flipsidecrypto.xyz")

    sql = f"""
    WITH tokens AS (
        SELECT 
            DECODED_LOG: token AS token,
            DECODED_LOG: tokenId AS tokenId,
            *
        FROM ethereum.core.ez_decoded_event_logs
        WHERE CONTRACT_ADDRESS = '{contract_address}'
        AND EVENT_NAME = 'TokenRegistered'
    )
    SELECT DISTINCT 
        token, 
        tokenId,
        COALESCE(p.symbol, token) AS symbol  -- Use metadata symbol if available
    FROM tokens t
    LEFT JOIN ethereum.price.ez_asset_metadata p ON t.token = p.token_address
    """

    query_result_set = flipside.query(sql)
    
    return auto_paginate_result(flipside, query_result_set)


def auto_paginate_result(flipside, query_result_set, page_size=100):
    """
    Automatically paginates through the API results and returns all records.
    
    :param flipside: The API client instance
    :param query_result_set: The initial query result set
    :param page_size: Number of records per page (default: 100)
    :return: A list of all records from paginated results
    """
    current_page_number = 1
    total_pages = 2  # Initial assumption
    all_records = []

    while current_page_number <= total_pages:
        results = flipside.get_query_results(
            query_result_set.query_id,
            page_number=current_page_number,
            page_size=page_size
        )
        
        if results.records:
            all_records.extend(results.records)
        
        total_pages = results.total_pages  # Update total pages from response
        current_page_number += 1

    return all_records
